# Инструкция по запуску AI-помощника в Google Colab:

1. В меню в верхней части страницы выберите *«Среда выполнения»* -> *«Сменить среду выполнения»* -> *«Графический процессор T4»*. Нажмите *«Сохранить»*.  

2. В меню выберите *«Выполнить все»*.  

3. Скачивание данных и установка необходимых пакетов займут несколько минут.  

4. Пролистайте страницу до вкладки *«Запуск модели»*.  

5. Введите данные во вкладке *«Пациент»* и нажмите *«Отправить»*. Ответ модели появится во вкладке *«Результаты»*

# Подготовка AI-помощника

In [ ]:
#@title Установка зависимостей
from IPython.display import clear_output
!pip install unsloth xformers trl peft accelerate bitsandbytes faiss-cpu langchain-community langchain-huggingface sentence-transformers PyPDF2
clear_output()

In [ ]:
#@title Скачивание данных, загрузка модели

import os
import gdown
import json
import torch
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

adapters_name = "llama_3_8b_full"
index_name = "russco_faiss"

adapters_id = "1c9N3vpGaqt7_HOIXnErj18NdJ9XCcW2i"
index_id = "18wRauo11wkQlPT6gMmy-6AlMLHWt77sG"
style_id = "19e3-ScfbU39nfucX-_NJwEhbhunYXdXF"

# Скачивание и распаковка адаптеров
gdown.download(id=adapters_id, output="adapters.zip", quiet=False)
!unzip -o -q adapters.zip -d adapters
print("Adapters downloaded and unzipped.")

# Скачивание и распаковка базы FAISS
gdown.download(id=index_id, output="index.zip", quiet=False)
!unzip -o -q index.zip -d index
print("FAISS index downloaded and unzipped.")

# Скачивание CSS + JS
gdown.download(id=style_id, output="style.txt", quiet=False)
with open("style.txt", "r") as file:
    css_js = file.read()
print("GUI style downloaded.")

# Загрузка основной модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "adapters/"+adapters_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# Загрузка модели эмбеддингов
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Загрузка базы данных FAISS
db = FAISS.load_local(
    "index/"+index_name,
    embeddings,
    allow_dangerous_deserialization=True
)
links_path = "index/"+index_name+"/links.json"
with open(links_path, 'r', encoding='utf-8') as file:
    links_dict = json.load(file)

clear_output()

In [ ]:
#@title Функции и классы

prompt_template = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.
{}
### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
"""

def get_context(treatment_plan):
    search_query = f"query: {treatment_plan}"
    docs = db.similarity_search(search_query, k=2)

    context = "\n---\n".join([
        f"Источник ({d.metadata.get('Header 2', 'Общее')}): {d.page_content}"
        for d in docs
    ])
    sources = [d.metadata.get('source', 'RUSSCO') for d in docs]
    return context, sources

def correct_treatment_plan(treatment_plan, context, doctor=True):
    added_text = "Формулируй ответ простыми словами, которые будут понятны пациенту.\n"
    if doctor:
        added_text = ""
    final_prompt = prompt_template.format(added_text, treatment_plan, context)
    inputs = tokenizer([final_prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    new_tokens = outputs[0][inputs.input_ids.shape[-1]:]
    result = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return result

class MapReducer:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.target_input_size = 1500

    def _generate(self, prompt, text, max_new_tokens=1024):
        formatted_prompt = f"### Инструкция:\n{prompt}\n\n### Данные:\n{text}\n\n### Ответ:\n"
        inputs = self.tokenizer([formatted_prompt], return_tensors="pt").to("cuda")
        outputs = self.model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, use_cache=True)
        return self.tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0].strip()

    def chunk_text(self, text, chunk_size=1500, overlap=200):
        tokens = self.tokenizer.encode(text)
        chunks = []
        for i in range(0, len(tokens), chunk_size - overlap):
            chunk_tokens = tokens[i : i + chunk_size]
            chunks.append(self.tokenizer.decode(chunk_tokens, skip_special_tokens=True))
            if i + chunk_size >= len(tokens): break
        return chunks

    def map_step(self, text_chunk):
        prompt = "Извлеки важные медицинские факты (диагнозы, симптомы, проведенное лечение, новые назначения) из этого фрагмента."
        return self._generate(prompt, text_chunk, max_new_tokens=500)

    def reduce_step(self, combined_summaries, is_final=False):
        if is_final:
            prompt = "На основе сжатых данных составь финальный анамнез пациента. Отдели уже проведенное лечение от планируемого."
        else:
            prompt = "Объедини и сожми следующие медицинские сводки, удаляя дубликаты и сохраняя важные клинические детали."
        return self._generate(prompt, combined_summaries, max_new_tokens=500)

    def process(self, long_history):
        chunks = self.chunk_text(long_history)
        summaries = [self.map_step(c) for c in chunks]

        current_text = "\n\n".join(summaries)
        while len(self.tokenizer.encode(current_text)) > self.target_input_size:
            new_summaries = []
            for i in range(0, len(summaries), 3):
                group = "\n\n".join(summaries[i:i+3])
                new_summaries.append(self.reduce_step(group, is_final=False))
            summaries = new_summaries
            current_text = "\n\n".join(summaries)

        return self.reduce_step(current_text, is_final=True)

reducer = MapReducer(model, tokenizer)

In [ ]:
#@title Создание интерфейса

import ipywidgets as widgets
from IPython.display import display, Javascript
import base64
from google.colab import files
import PyPDF2
import io
import requests
import random

class MedicalAssistant:

    def __init__(self):
        self.setup_ui()
        display(widgets.HTML(css_js))

    def setup_ui(self):
        title = widgets.HTML(
            value="<h1 style='color: #2c3e50; text-align: center;'>🏥 Медицинский AI ассистент</h1>"
        )

        self.role = widgets.Dropdown(
            options=[('Пациент', 'patient'), ('Врач', 'doctor')],
            value='doctor',
            description='Вы:',
            style={'description_width': 'initial'}
        )

        tab = widgets.Tab()

        self.patient_info = widgets.Textarea(
            value='',
            placeholder='Введите информацию о пациенте...',
            description='Информация:',
            layout=widgets.Layout(width='100%', height='150px'),
            style={'description_width': 'initial'}
        )

        self.upload_button = widgets.Button(
            description='Загрузить PDF',
            button_style='primary',
            icon='upload',
        )
        self.upload_button.on_click(self.upload_pdf)

        self.pdf_output = widgets.Output()

        self.submit_patient_info = widgets.Button(
            description='Отправить',
            button_style='success',
            icon='check',
        )
        self.submit_patient_info.on_click(self.process_patient_info)

        patient_tab = widgets.VBox([
            widgets.HTML("<h3>📋 Информация о пациенте</h3>"),
            self.role,
            self.patient_info,
            widgets.HBox([self.upload_button]),
            self.pdf_output,
            self.submit_patient_info
        ])

        self.support_email = widgets.Text(
            value='',
            placeholder='Ваш email для ответа',
            description='Ваш email:',
            layout=widgets.Layout(width='100%')
        )

        self.support_message = widgets.Textarea(
            value='',
            placeholder='Опишите вашу проблему или вопрос...',
            description='Сообщение:',
            layout=widgets.Layout(width='100%', height='150px'),
            style={'description_width': 'initial'}
        )

        self.support_subject = widgets.Text(
            value='',
            placeholder='Тема сообщения',
            description='Тема:',
            layout=widgets.Layout(width='100%')
        )

        self.send_support = widgets.Button(
            description='Отправить',
            button_style='warning',
            icon='envelope'
        )
        self.send_support.on_click(self.send_support_email)

        self.support_output = widgets.Output()

        support_tab = widgets.VBox([
            widgets.HTML("<h3>📧 Техническая поддержка</h3>"),
            self.support_email,
            self.support_subject,
            self.support_message,
            self.send_support,
            self.support_output
        ])

        self.results_output = widgets.Output()

        results_tab = widgets.VBox([
            widgets.HTML("<h3>📊 Результаты обработки</h3>"),
            self.results_output
        ])

        tab.children = [patient_tab, support_tab, results_tab]
        tab.set_title(0, '👤 Пациент')
        tab.set_title(1, '🆘 Поддержка')
        tab.set_title(2, '📊 Результаты')

        display(widgets.VBox([title, tab]))

    def upload_pdf(self, b):
        with self.pdf_output:
            clear_output(wait=True)
            uploaded = files.upload()

            for filename in uploaded.keys():
                pdf_content = uploaded[filename]

                try:
                    pdf_file = io.BytesIO(pdf_content)
                    pdf_reader = PyPDF2.PdfReader(pdf_file)

                    text = ""
                    for page in pdf_reader.pages:
                        text += page.extract_text()

                    self.patient_info.value = text

                    print(f"✅ Файл {filename} успешно загружен и обработан")
                    print(f"📄 Извлечено {len(pdf_reader.pages)} страниц")

                except Exception as e:
                    print(f"❌ Ошибка при обработке PDF: {str(e)}")

    def process_patient_info(self, b):
        with self.results_output:
            clear_output(wait=True)

            role_text = "Врач" if self.role.value == "doctor" else "Пациент"
            doctor = True if self.role.value == "doctor" else False
            info = self.patient_info.value

            if not info.strip():
                print("⚠️ Пожалуйста, введите информацию о пациенте")
                return

            print(f"👤 Роль: {role_text}")
            print(f"📝 Полученная информация:\n{info}\n")

            if len(tokenizer.encode(info)) > 700:
                print("⏱️ Запрос слишком большой, сжатие займет дополнительное время...")
                info = reducer.process(info)
            print("🤖 AI ассистент обрабатывает запрос...")
            context, sources = get_context(info)
            corrections = correct_treatment_plan(info, context, doctor)

            print(f"🤖 Ответ AI ассистента:\n{corrections}")
            print("\n📖 Источники:")
            link_list = []
            for source in sources:
                source_link = links_dict[source]
                link = widgets.HTML(f"<a href='{source_link}' target='_blank'>{source}</a>")
                link_list.append(link)
            display(widgets.VBox(link_list))

    def send_support_email(self, b):
        with self.support_output:
            clear_output(wait=True)

            url = "https://docs.google.com/forms/d/e/1FAIpQLSccA0WN_YDAFvotE-oVgQiKf_nNpAAXwvHj8Vh3WILbd-MFnQ/formResponse"
            form_data = {
                'entry.1968432315': self.support_email.value,
                'entry.189793766': self.support_subject.value,
                'entry.1519388799': self.support_message.value
            }

            if not self.support_message.value:
                print("⚠️ Пожалуйста, введите сообщение")
                return
            else:
                response = requests.post(url, data=form_data)

            if response.status_code == 200:
                print("✅ Сообщение отправлено в техподдержку")
                print("📧 Мы свяжемся с вами в ближайшее время")
            else:
                print(f"❌ Ошибка при отправке: {response.status_code}")

# Запуск модели

In [ ]:
#@title Запуск модели
assistant = MedicalAssistant()